# CalCOFI

Over 60 years of oceanographic data from California Cooperative Oceanic Fisheries Investigations (CalCOFI).

Dataset downloaded from Kaggle: https://www.kaggle.com/datasets/sohier/calcofi

Column info: https://calcofi.org/data/oceanographic-data/bottle-database/

# Table of Contents<a id="toc"></a>

1. [Question](#question)
2. [EDA and Data Preparation](#eda)
    1. [Load data](#loaddata)
    2. [Remove columns that are not needed](#delete1)
    3. [Handle quality values](#delete2)
    4. [Remove columns with low data count](#delete3)
    5. [Remove null values from chlorophyll column](#dropna)
    6. [Remove duplicate columns](#delete4)
    7. [Lasso regression](#lasso)
        1. [Fill null values](#fillna)
        2. [Fitting the model](#lassofitting)
        3. [Using the lasso results](#delete5)
    8. [Deleting bad quality data](#delete2_2)
        4. [Resuming use of lasso results](#resumelasso)
    9. [Feature correlation](#correlation)
    10. [Make sure column values are the right type](#checkschema)
    11. [Summary Statistics](#summarystatistics)
3. [Feature Selection and Target Cleaning](#featuretarget)
    12. [Distribution of data for cholorophyll levels, summary stats](#summarystats)
    13. [Define cholorophyll category levels: low, medium, high?](#lowmediumhigh)
    14. [Map cholorophyll levels](#maplevels)
4. [Models](#models)
    15. Setup
        5. [Assemble Feature Vector](#assemble)
        6. [Split Data: Train, Test, Validation, 80, 10, 10](#split)
        7. [Scale Data](#scale)
        8. [Check for Imblance in datasets](#checkimbalance)
    16. [Logistic Regresssion Model](#logisticregression)
        9. [Testing parameters](#lrparams)
        10. [Fit the model with optimal parameters](#lrfit)
        11. [Results](#lrresults)
    17. [Decision Tree](#decisiontree)
        12. [Testing parameters](#dtparams)
        13. [Fit the model with optimal parameters](#dtfit)
        14. [Results](#dtresults)
5. [Stop spark session](#stop)

# Questions<a id="question"></a>

Can we predict chlorophyll abundance without having to measure it?

If so, which chemicals have the highest influence on the predictions?

# EDA and Data Preparation<a id="eda"></a>

In [1]:
from pyspark.sql.functions import *

#### Load data<a id="loaddata"></a>

In [2]:
# Loading in spark
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[2]'), 
    ('spark.app.name', 'App Name')])
    
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-30 00:39:56,536 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.2.1'

In [3]:
# Read the data from hdfs
bottle = spark.read.csv('hdfs:///bottle.csv', header=True, inferSchema=True).cache()
maxRows = bottle.count()
print('There are', maxRows, 'rows in the initial dataframe')

2022-05-30 00:40:05,978 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


There are 864863 rows in the initial dataframe


In [4]:
# View the inferred schema
print('There are', len(bottle.columns), 'columns in the original dataframe')
bottle.printSchema()

There are 74 columns in the original dataframe
root
 |-- Cst_Cnt: integer (nullable = true)
 |-- Btl_Cnt: integer (nullable = true)
 |-- Sta_ID: string (nullable = true)
 |-- Depth_ID: string (nullable = true)
 |-- Depthm: integer (nullable = true)
 |-- T_degC: double (nullable = true)
 |-- Salnty: double (nullable = true)
 |-- O2ml_L: double (nullable = true)
 |-- STheta: double (nullable = true)
 |-- O2Sat: double (nullable = true)
 |-- Oxy_µmol/Kg: double (nullable = true)
 |-- BtlNum: integer (nullable = true)
 |-- RecInd: integer (nullable = true)
 |-- T_prec: integer (nullable = true)
 |-- T_qual: integer (nullable = true)
 |-- S_prec: integer (nullable = true)
 |-- S_qual: integer (nullable = true)
 |-- P_qual: integer (nullable = true)
 |-- O_qual: integer (nullable = true)
 |-- SThtaq: integer (nullable = true)
 |-- O2Satq: integer (nullable = true)
 |-- ChlorA: double (nullable = true)
 |-- Chlqua: integer (nullable = true)
 |-- Phaeop: double (nullable = true)
 |-- Phaqua: i

In [5]:
# See a snippet of what this dataframe looks like
bottle.show(2)

+-------+-------+-----------+--------------------+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+-----+------+-------+------+------+----+----+----+----+----+----+-------------------+
|Cst_Cnt|Btl_Cnt|     Sta_ID|            Depth_ID|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|BtlNum|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3uM|NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|IncTim|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_NH4|R_CH

***
[(Return to Table of Contents)](#toc)

### Remove columns that are not needed<a id="delete1"></a>

Removing the four string columns because they aren't useful for our purposes
* Sta_ID: Line and Station
* Depth_ID: Uses the Cast_ID prefix ([Century]-[Year][Month][ShipCode]-[CastType][Julian Day]-[CastTime]-[Line][Sta]) but adds three additional variables: [Depth][Bottle]-[Rec_Ind]
* IncTim: Elapsed incubation time of the primary productivity experiment
* DIC Quality Comment: Quality Comment

Also removing the Cast and Bottle counts, which are essentially indexes (identifiers)
* 'Cst_Cnt': Auto-numbered Cast Count - all casts consecutively numbered. 1 is first station done
* 'Btl_Cnt': Auto-numbered Bottle count- all bottles ever sampled, consecutively numbered
* 'BtlNum': Bottle Number

In [6]:
# Dropping unneeded columns and viewing two rows of the resulting dataframe
deleteList1 = ['Sta_ID','Depth_ID','IncTim','DIC Quality Comment','Cst_Cnt','Btl_Cnt','BtlNum']
bottle = bottle.drop(*deleteList1)

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
bottle.show(2, truncate=False)

There are now 67 columns and 864863 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+-----+------+-------+------+------+----+----+----+----+----+----+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3uM|NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_NH4|R_CHLA|R_PHAEO|R_PRES|R_SAMP|DIC1|DIC2|TA1 |TA2 |pH2 |pH1 |
+------+------+------+------+------+-----+-------

***
[(Return to Table of Contents)](#toc)

### Handle quality values<a id="delete2"></a>

These columns indicate the quality of the measurements taken. Eventually we will delete these columns because we want to use the quantity measurements for our final model, but for now we keep these columns because we'll use their code for "badly recorded data" to delete those badly recorded measurements.
* T_qual: Temperature Quality Code
* S_qual: Salinity Quality Code
* P_qual: Pressure Quality Code
* O_qual: Oxygen Quality Code
* 'O2Satq': Oxygen Saturation Quality Code
* 'Chlqua': Chlorophyll-a Quality Code
* 'Phaeop': Phaeophytin Quality Code
* 'Phaqua': Phosphate Quality Code
* 'PO4uM': Salinity Quality Code
* 'PO4q': Phosphate Quality Code
* 'SiO3qu': Quality Code
* 'NO2q': Quality Code
* 'NO3q': Nitrate Quality Code
* 'NH3q': Ammonium Quality Code
* 'C14A1q': 14C As1 Quality Code
* 'C14A2q': 14C As2 Quality Code
* 'DarkAq': 14C Assimilation Dark Bottle Quality Code
* 'MeanAq': Mean 14C Assimilation Quality Code

When the quality code = 8, the recorded measurement is considered badly measured. Eventually we will use these 8s to delete rows from the dataframe, but first we'd like to use Lasso regression to find the most important columns of which these quality columns can be used for.

In [7]:
# Create the list of quality columns, but don't delete yet
deleteList2 = ['T_qual','S_qual','P_qual','O_qual','O2Satq','Chlqua', 'Phaqua','PO4q',
               'SiO3qu','NO2q','NO3q','NH3q','C14A1q','C14A2q','DarkAq','MeanAq']

***
[(Return to Table of Contents)](#toc)

### Remove columns with low data count<a id="delete3"></a>

We start by counting the number of NaNs and nulls in each column and reporting them in a new dataframe. Then the columns with less than 200,000 non-nulls are deleted.

In [8]:
# Counting the number of null/NaN rows per column and outputting that in a new dataframe
def getNullCounts(df):
    return df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

nullCounter = getNullCounts(bottle)
nullCounter.show()

+------+------+------+------+------+------+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+------+-------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta| O2Sat|Oxy_µmol/Kg|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua| PO4uM|  PO4q|SiO3uM|SiO3qu| NO2uM|  NO2q| NO3uM|  NO3q| NH3uM| NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|  R_O2|R_O2Sat|R_SIO3| R_PO4| R_NO3| R_NO2| R_NH4|R_CHLA|R_PHAEO|R_PRES|R_SAMP|  DIC1|  DIC2|   TA1|   TA2|   pH2|   pH1|
+------+------+------+------+-

In [9]:
# Deleting columns with less than 200000 non-nulls
thresh = maxRows - 200000
deleteList3 = []
for value in nullCounter.columns:
    if nullCounter.filter(nullCounter[value] > thresh).select(nullCounter[value]).collect():
        deleteList3.append(value)
bottle = bottle.drop(*deleteList3)

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
bottle.show(2, truncate=False)

There are now 45 columns and 864863 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|0     |10.5  

***
[(Return to Table of Contents)](#toc)

### Remove null values from chlorophyll column<a id="dropna"></a>

Because this is the target column, we can only use the non-null rows. We also get rid of the duplicate chlorophyll column `ChlorA`, keeping `R_CHLA` because it has less non-null rows (though only by a few).

In [10]:
# Illustrating that we have two target columns that are essentially duplicates of each other
bottle.corr('ChlorA','R_CHLA')

0.9999995087108816

Because we only need one of these columns, we can delete the other. Then we drop all null rows of the R_CHLA column since it's our target.

In [11]:
# Looking at which of the two target columns have more NaNs in order to select which to delete
nullCounter.select(*['ChlorA','R_CHLA']).show()

+------+------+
|ChlorA|R_CHLA|
+------+------+
|639591|639587|
+------+------+



In [12]:
# Delete the ChlorA column and drop all NaN rows in the R_CHLA column
bottle = bottle.drop('ChlorA')
bottle = bottle.dropna(subset='R_CHLA')

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
bottle.show(2, truncate=False)

There are now 44 columns and 225276 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|0     |19.23 |34.491|5.46  |24.575

***
[(Return to Table of Contents)](#toc)

### Remove duplicate columns<a id="delete4"></a>

As we can see from the column above, there are some columns that are essentially duplicates of each other but in different units of measurement. Of the remaining columns, here are their descriptions that will help us determine which features to compare for potential deletion:

In [13]:
# See what columns we still have

attributes = bottle.columns
for element in deleteList2:
    if element in attributes:
        attributes.remove(element)

print(attributes)
del attributes

['Depthm', 'T_degC', 'Salnty', 'O2ml_L', 'STheta', 'O2Sat', 'Oxy_µmol/Kg', 'RecInd', 'T_prec', 'S_prec', 'Phaeop', 'PO4uM', 'SiO3uM', 'NO2uM', 'NO3uM', 'R_Depth', 'R_TEMP', 'R_POTEMP', 'R_SALINITY', 'R_SIGMA', 'R_SVA', 'R_DYNHT', 'R_O2', 'R_O2Sat', 'R_SIO3', 'R_PO4', 'R_NO3', 'R_NO2', 'R_CHLA', 'R_PHAEO', 'R_PRES']


Chlorophyll
* 'ChlorA': Acetone extracted chlorophyll-a measured fluorometrically
* 'R_CHLA': Reported Chlorophyll-a (micrograms per liter)

Depth
* 'Depthm': Depth in meters
* 'R_Depth': Reported Depth (from pressure) in meters

Water density
* 'STheta': Potential Density of Water
* 'R_SIGMA': Reported Potential Density of water

Silicate
* 'SiO3uM': Micromoles Silicate per liter of seawater
* 'R_SIO3': Reported Silicate Concentration

Nitrite
* 'NO2uM': Micromoles Nitrite per liter of seawater
* 'R_NO2': Reported Nitrite Concentration

Nitrate
* 'NO3uM': Micromoles Nitrate per liter of seawater
* 'R_NO3': Reported Nitrate Concentration

Salinity
* 'Salnty': Practical Salinity Scale, 1978 (UNESCO, 1981a); Salinity of water
* 'R_SALINITY': Reported Salinity (from Specific Volume Anomoly, M³/Kg)

O2 saturation
* 'O2Sat': Percent Saturation; Oxygen Saturation
* 'R_O2Sat': Percent	Reported Oxygen Saturation

Phaeophytin
* 'Phaeop': Micrograms Phaeophytin per liter of seawater
* 'R_PHAEO': Reported Phaeophytin

Oxygen
* 'O2ml_L': Oxygen in mL/L; Milliliters of dissolved oxygen per Liter seawater
* 'Oxy_µmol/Kg': Oxygen in micro moles per kilogram of seawater
* 'R_O2': Reported milliliters of oxygen per liter of seawater

Temperature
* 'T_degC': Temperature of Water
* 'R_TEMP': Reported Temperature (Celsius)
* 'R_POTEMP': Reported Potential Temperature (Celsius)

Other
* 'S_prec': Salinity Units of Precision
* 'T_prec': Temperature Units of Precision
* 'RecInd': Record Indicator
* 'R_SVA': Reported Specific Volume Anomaly
* 'R_DYNHT': Reported Dynamic Height
* 'R_PO4': Reported Phosphate Concentration
* 'R_PRES': Pressure in decibars

In [14]:
# Setting up a function that will streamline the comparing process
nullCounter = getNullCounts(bottle)

def psudoDuplicateCheck(features):
    nullCounter.select(*features).show()

    if len(features)==2:
        print('correlation:', bottle.corr(features[0],features[1]))
    elif len(features)==3:
        print('1&2 correlation:', bottle.corr(features[0],features[1]))
        print('2&3 correlation:', bottle.corr(features[1],features[2]))
        print('1&3 correlation:', bottle.corr(features[0],features[2]))

In [15]:
nullCounter.show()

+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+------+------+------+-----+------+-----+------+-----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaeop|Phaqua|PO4uM|  PO4q|SiO3uM|SiO3qu|NO2uM|  NO2q|NO3uM|  NO3q| NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+------+------+------+-----+------+-----+------+-----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|     0|  3088|  3433|  3610|  3781| 4500|       4502| 

In [16]:
Depths = ['Depthm','R_Depth']
print('Depths NaN count:')
psudoDuplicateCheck(Depths)

WD = ['STheta','R_SIGMA']
print('\nWater density NaN count:')
psudoDuplicateCheck(WD)

Silicate = ['SiO3uM','R_SIO3']
print('\nSilicate NaN count:')
psudoDuplicateCheck(Silicate)

Nitrite = ['NO2uM','R_NO2']
print('\nNitrite NaN count:')
psudoDuplicateCheck(Nitrite)

Nitrate = ['NO3uM','R_NO3']
print('\nNitrate NaN count:')
psudoDuplicateCheck(Nitrate)

Salinity = ['Salnty','R_SALINITY']
print('\nSalinity NaN count:')
psudoDuplicateCheck(Salinity)

Saturation = ['O2Sat','R_O2Sat']
print('\nO2 Saturation NaN count:')
psudoDuplicateCheck(Saturation)

Phaeophytin = ['Phaeop', 'R_PHAEO']
print('\nPhaeophytin NaN count:')
psudoDuplicateCheck(Phaeophytin)

Oxygen = ['O2ml_L','Oxy_µmol/Kg','R_O2']
print('\nOxygen NaN count:')
psudoDuplicateCheck(Oxygen)

Temperature = ['T_degC','R_TEMP','R_POTEMP']
print('\nTemperature NaN count:')
psudoDuplicateCheck(Temperature)

del nullCounter

Depths NaN count:
+------+-------+
|Depthm|R_Depth|
+------+-------+
|     0|      0|
+------+-------+

correlation: 0.9999999949168986

Water density NaN count:
+------+-------+
|STheta|R_SIGMA|
+------+-------+
|  3781|   3655|
+------+-------+

correlation: 0.9775619506024495

Silicate NaN count:
+------+------+
|SiO3uM|R_SIO3|
+------+------+
|  8947|  8941|
+------+------+

correlation: 0.9999991048864016

Nitrite NaN count:
+-----+-----+
|NO2uM|R_NO2|
+-----+-----+
|14951|14945|
+-----+-----+

correlation: 0.9999753963583337

Nitrate NaN count:
+-----+-----+
|NO3uM|R_NO3|
+-----+-----+
| 9573| 9567|
+-----+-----+

correlation: 0.9999998732099501

Salinity NaN count:
+------+----------+
|Salnty|R_SALINITY|
+------+----------+
|  3433|      3433|
+------+----------+

correlation: 0.9999999893049997

O2 Saturation NaN count:
+-----+-------+
|O2Sat|R_O2Sat|
+-----+-------+
| 4500|   4352|
+-----+-------+

correlation: 0.9903641742840068

Phaeophytin NaN count:
+------+-------+
|Phaeo

We now delete the columns with fewer null if they have a correlation coeffecients of 0.97 or higher.

In the case of a tie, we can refer to the pattern that is very apparent in no-tie cases, which is that the 'reported' (columns starting with `R_`) have the higher non-null count. Therefore if two columns have equal null counts and high enough correlation, we delete the not-"reported" column.

In [17]:
# Deleting columns that are duplicates of some other column
deleteList4 = ['Depthm','STheta','SiO3uM','NO2uM','NO3uM','Salnty','O2Sat',
               'O2ml_L','Oxy_µmol/Kg','T_degC','R_POTEMP', 'Phaeop']
bottle = bottle.drop(*deleteList4)

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
bottle.show(2, truncate=False)

There are now 32 columns and 225276 rows
+------+------+------+------+------+------+------+-----+----+------+----+----+----+------+------+------+------+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaqua|PO4uM|PO4q|SiO3qu|NO2q|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+------+------+-----+----+------+----+----+----+------+------+------+------+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|3     |2     |3     |9     |null  |null  |null  |null |9   |9     |9   |null|9   |9     |9     |9     |9     |0.0    |19.23 |34.491    |24.57  |335.3|0.0    |5.46|103.9  |null  |null |1.3  |null |0.64  |0.47   |0     |
|3     |2     |3     |9     |null  |null  |null  |null |9   |9     |9   |null|9

***
[(Return to Table of Contents)](#toc)

### Lasso regression<a id="lasso"></a>

As we lead up to performing Lasso regression, we'll need a dummy dataframe to permanently mess with. We create `bottleTest` for this and remove the quality columns in preparation.

In [18]:
# Create temp dataframe and remove Quality Code columns from temp dataframe for lasso
bottleTest = bottle.alias('bottleTest')
bottleTest = bottleTest.drop(*deleteList2)

##### Fill null values<a id="fillna"></a>

In [19]:
# Using the function from PA3
from pyspark.ml.feature import Imputer

def fill_na(df, strategy):    
    imputer = Imputer(
        strategy=strategy,
        inputCols=df.columns, 
        outputCols=['{}_imputed'.format(c) for c in df.columns]
    )
    
    new_df = imputer.fit(df).transform(df)
    
    # Select the newly created columns with all filled values
    new_df = new_df.select([c for c in new_df.columns if 'imputed' in c])
    
    for col in new_df.columns:
        new_df = new_df.withColumnRenamed(col, col.split('_imputed')[0])
        
    return new_df

In [20]:
# Filling in the remaining null rows with the mean of the column and saving this newly
#  filled in dataframe as a new variable
bottleTest = fill_na(bottleTest, 'mean')

print('There are still', len(bottleTest.columns), 'columns and', bottleTest.count(), 'rows')
bottleTest.show(2, truncate=False)

There are still 19 columns and 225276 rows
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+------+-------+------+
|RecInd|T_prec|S_prec|PO4uM             |R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3            |R_PO4             |R_NO3|R_NO2              |R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+------+-------+------+
|3     |2     |3     |0.9506321528926772|0.0    |19.23 |34.491    |24.57  |335.3|0.0    |5.46|103.9  |11.529812096979274|0.9506145966112527|1.3  |0.05539378408325813|0.64  |0.47   |0     |
|3     |2     |3     |0.9506321528926772|10.0   |19.22 |34.492    |24.57  |335.3|0.03   |5.46|103.9  |11.529812096979274|0.9506145966112527|1.9  |0.05539378408325813|0.66  |0.38   |10    |
+------+----

In [21]:
# Double checking that all null values are filled
getNullCounts(bottleTest).show()

+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|RecInd|T_prec|S_prec|PO4uM|R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|     0|     0|     0|    0|      0|     0|         0|      0|    0|      0|   0|      0|     0|    0|    0|    0|     0|      0|     0|
+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+



##### Fitting the model<a id="lassofitting"></a>

To better estimate what columns are most important to our model, we use a lasso regression. Because spark LassoWithSGD has depricated, we use LinearRegression with `elasticNetParam = 1.0` for Lasso regression equivalent.

In order to use this, `label` and `features` columns need to be specified. We rename `R_CHLA` to `label` and create a features column with VectorAssembler.

In [22]:
# Rename the target chlorophyll column to 'label'
bottleTest = bottleTest.withColumnRenamed('R_CHLA', 'label')

In [23]:
# Creating 'features' column
from pyspark.ml.feature import VectorAssembler

# (interm step) make list of column names other than 'label,' AKA make a list of the features
features = bottleTest.columns
features.remove('label')

bottleTest = VectorAssembler(outputCol='features').setInputCols(features).transform(bottleTest)

bottleTest.show(2, truncate=False)

+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+-----+-------+------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|RecInd|T_prec|S_prec|PO4uM             |R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3            |R_PO4             |R_NO3|R_NO2              |label|R_PHAEO|R_PRES|features                                                                                                                                              |
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+-----+-------+------+---------------------------------------------------------------------------------------------------------------------------------------

In [24]:
# Applying lasso regression where regParam= 0.0025 in place of alpha
from pyspark.ml.regression import LinearRegression

lasso = LinearRegression(elasticNetParam = 1.0, regParam=0.0025, solver='normal',
                      maxIter=1000, standardization=True)
lassoModel = lasso.fit(bottleTest)

2022-05-30 00:40:32,291 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
2022-05-30 00:40:32,295 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


##### Using the lasso results<a id="delete5"></a>

Now that we've fit the model and some columns with nonzero coeffecients, we create a list of the 'relevant' and 'not useful' features.

In [25]:
# Find the 'relevant' and 'not useful' features, as according to the lasso regresion when regParam=0.0025

coeff = lassoModel.coefficients
usefulFeatures = []
deleteList5 = []

print('(For-loop of', len(coeff), 'iterations)')
for i in range(len(coeff)):
    if coeff[i] != 0:
        usefulFeatures.append(features[i])
    else:
        deleteList5.append(features[i])

print('When regParam='+str(lasso.getRegParam())+', this lasso model indicates the most useful columns to predicting chlorophyll are:',
      usefulFeatures)
del usefulFeatures

(For-loop of 18 iterations)
When regParam=0.0025, this lasso model indicates the most useful columns to predicting chlorophyll are: ['S_prec', 'R_TEMP', 'R_SALINITY', 'R_DYNHT', 'R_O2', 'R_O2Sat', 'R_SIO3', 'R_NO3', 'R_NO2', 'R_PHAEO']


In [26]:
# Moving back to dataframe before nulls were filled with means, drop the 'not useful' features
bottle = bottle.drop(*deleteList5)

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')

There are now 24 columns and 225276 rows


In [27]:
lowThreshold = .1
medThreshold = 1

bottle.withColumn(
    'Target',
     when((col('R_CHLA') < lowThreshold), 0)\
    .when((col('R_CHLA').between(lowThreshold, medThreshold)), 1)\
    .when((col('R_CHLA') > medThreshold), 2)\
    .otherwise(10)
).groupby('Target').count().show()

+------+------+
|Target| count|
+------+------+
|     1|124049|
|     2| 19665|
|     0| 81562|
+------+------+



In [28]:
getNullCounts(bottle).show()

+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|S_prec|P_qual|O2Satq|Chlqua|Phaqua|  PO4q|SiO3qu|  NO2q|  NO3q| NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_TEMP|R_SALINITY|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_NO3|R_NO2|R_CHLA|R_PHAEO|
+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|  3433|136270|214052|225067|225059|214865|216273|208998|213376|40530| 15560| 15542| 23697| 23698|  3088|      3433|   3527|3610|   4352|  8941| 9567|14945|     0|      5|
+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+----------+-------+----+-------+------+-----+-----+------+-------+



Before we can fill the 'most useful' column's (the remaining columns) null entries with column means, we need to use and then delete the quality columns.

***
[(Return to Table of Contents)](#toc)

### Deleting bad quality data<a id="delete2_2"></a>

Now that we have the most useful columns from the Lasso model, we can prune the dataframe of poorly recorded data based on the quality columns.

In [32]:
# Creating a map/2D-list of the quality columns with the measurement columns they correspond to
# in order to delete the low-quality records from our current bottle dataframe
qualityColWithRCol = [deleteList2,
                      ['R_TEMP', 'R_SALINITY', 'R_PRES', 'R_O2', 'R_O2Sat', 'R_CHLA', 'Phaeop', 'R_PO4' \
                      'R_SIO3', 'R_NO2', 'R_NO3', 'R_Nuts', 'C14As1', 'C14As2', 'DarkAp', 'MeanAs']]

In [33]:
# Print out the number of bad-quality recordings per column in the bottle dataframe
for i in range(len(qualityColWithRCol[0])):
    colName = qualityColWithRCol[0][i]
    if(colName in bottle.columns):
        numBadData = bottle.filter(bottle[qualityColWithRCol[0][i]] == 8).count()
        print("Number of bad quality data for column " + colName + ":", numBadData)

# Dropping bad quality rows of quality columns
for i in range(len(qualityColWithRCol[0])):
    colName = qualityColWithRCol[0][i]
    if(colName in bottle.columns):
        bottle = bottle.filter((bottle[qualityColWithRCol[0][i]] != 8) | \
                               (bottle[qualityColWithRCol[0][i]].isNull()))

del qualityColWithRCol

Number of bad quality data for column P_qual: 0
Number of bad quality data for column O2Satq: 0
Number of bad quality data for column Chlqua: 0
Number of bad quality data for column Phaqua: 0
Number of bad quality data for column PO4q: 0
Number of bad quality data for column SiO3qu: 0
Number of bad quality data for column NO2q: 0
Number of bad quality data for column NO3q: 0
Number of bad quality data for column NH3q: 0
Number of bad quality data for column C14A1q: 0
Number of bad quality data for column C14A2q: 0
Number of bad quality data for column DarkAq: 0
Number of bad quality data for column MeanAq: 0


In [31]:
# Now we can get rid of the quality columns since they've served their purpose
bottle = bottle.drop(*deleteList2)

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
bottle.show(2, truncate=False)

There are now 11 columns and 222293 rows
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_NO3|R_NO2|R_CHLA|R_PHAEO|
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|3     |19.23 |34.491    |0.0    |5.46|103.9  |null  |1.3  |null |0.64  |0.47   |
|3     |19.22 |34.492    |0.03   |5.46|103.9  |null  |1.9  |null |0.66  |0.38   |
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
only showing top 2 rows



##### Resuming use of lasso results<a id="resumelasso"></a>

In [32]:
getNullCounts(bottle).show()

+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_NO3|R_NO2|R_CHLA|R_PHAEO|
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|  3433|  3088|      3433|   3527|3607|   4231|  8926| 9549|14913|     0|      5|
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+



In [33]:
# Moving back to original dataframe, fill the 'most useful' column's (the remaining
#  columns) null entries with means
bottle = bottle.dropna()

print('There are now', len(bottle.columns), 'columns and', bottle.count(), 'rows')
getNullCounts(bottle).show()

There are still 11 columns and 222293 rows
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_NO3|R_NO2|R_CHLA|R_PHAEO|
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+
|     0|     0|         0|      0|   0|      0|     0|    0|    0|     0|      0|
+------+------+----------+-------+----+-------+------+-----+-----+------+-------+



In [34]:
# Now that we're done with this temporary dataframe and the Lasso model, we can delete them
del bottleTest, lasso, lassoModel

***
[(Return to Table of Contents)](#toc)

### Feature correlation<a id="correlation"></a>

Finding columns that are similar and analyse if we can get rid of null columns without removal of variance.

In [35]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [36]:
columns = ['col1', 'col2', 'correlation']
schema = StructType([
  StructField('col1', StringType(), False),
  StructField('col2', StringType(), False),
  StructField('correlation', IntegerType(), False)
  ])
highCorrs = spark.createDataFrame(spark.sparkContext.emptyRDD() ,schema)
rangeColumns = range(len(bottle.columns))
for i in rangeColumns:
    for j in rangeColumns:
        if(i != j):
            correlation = bottle.corr(bottle.columns[i], bottle.columns[j])
            if(correlation > .9):
                newCorr = spark.createDataFrame([(bottle.columns[i], bottle.columns[j], correlation)], columns)
                highCorrs = highCorrs.union(newCorr)
            else: 
                 continue;

In [37]:
highCorrs.show()

+-------+-------+------------------+
|   col1|   col2|       correlation|
+-------+-------+------------------+
|   R_O2|R_O2Sat|0.9873734309503178|
|R_O2Sat|   R_O2|0.9873734309503178|
| R_SIO3|  R_NO3|0.9722648506285719|
|  R_NO3| R_SIO3|0.9722648506285719|
+-------+-------+------------------+



Because R_O2 is highly correlated with R_O2Sat and are very similar metrics, we are dropping the R_O2 column.  R_SIO3 is highly correlated with R_NO3 so we are dropping the R_SIO3 column and renaming the R_SIO3 to include the R_NO3 in the title to remember it contributes similar variance.

In [38]:
bottle = bottle.drop('R_O2')
bottle.withColumnRenamed('R_O2Sat', 'R_O2, R_O2Sat')
bottle = bottle.drop('R_SIO3')
bottle.withColumnRenamed('R_NO3', 'R_SIO3, R_NO3')
bottle.show(2)

+------+------+----------+-------+-------+-----+-------------------+------+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2Sat|R_NO3|              R_NO2|R_CHLA|R_PHAEO|
+------+------+----------+-------+-------+-----+-------------------+------+-------+
|     3| 19.23|    34.491|    0.0|  103.9|  1.3|0.05597931333790767|  0.64|   0.47|
|     3| 19.22|    34.492|   0.03|  103.9|  1.9|0.05597931333790767|  0.66|   0.38|
+------+------+----------+-------+-------+-----+-------------------+------+-------+
only showing top 2 rows



***
[(Return to Table of Contents)](#toc)

### Make sure column values are the right type<a id="checkschema"></a>

In [39]:
# Check the dtypes of each column
bottle.printSchema()

root
 |-- S_prec: integer (nullable = true)
 |-- R_TEMP: double (nullable = true)
 |-- R_SALINITY: double (nullable = true)
 |-- R_DYNHT: double (nullable = true)
 |-- R_O2Sat: double (nullable = true)
 |-- R_NO3: double (nullable = true)
 |-- R_NO2: double (nullable = true)
 |-- R_CHLA: double (nullable = true)
 |-- R_PHAEO: double (nullable = true)



In [40]:
# Row count, and comparison to original row count
print('There are now', len(bottle.columns), 'columns and', bottle.count(),
      'rows (which is', maxRows-bottle.count(), 'less than the original row count)')

There are now 9 columns and 222293 rows (which is 642570 less than the original row count)


***
[(Return to Table of Contents)](#toc)

### Summary statistics<a id="summarystatistics"></a>

In [41]:
# See the detailed numerical description of the current dataframe
bottle.describe().show(vertical=True)

-RECORD 0-------------------------
 summary    | count               
 S_prec     | 222293              
 R_TEMP     | 222293              
 R_SALINITY | 222293              
 R_DYNHT    | 222293              
 R_O2Sat    | 222293              
 R_NO3      | 222293              
 R_NO2      | 222293              
 R_CHLA     | 222293              
 R_PHAEO    | 222293              
-RECORD 1-------------------------
 summary    | mean                
 S_prec     | 2.9838906308340793  
 R_TEMP     | 12.902264391779196  
 R_SALINITY | 33.506654879831125  
 R_DYNHT    | 0.21082594644507371 
 R_O2Sat    | 80.94547605727023   
 R_NO3      | 9.64867587335387    
 R_NO2      | 0.05597931333791566 
 R_CHLA     | 0.45214100309038985 
 R_PHAEO    | 0.19925708990137597 
-RECORD 2-------------------------
 summary    | stddev              
 S_prec     | 0.12589649992341595 
 R_TEMP     | 3.035793082781941   
 R_SALINITY | 0.31335388232128686 
 R_DYNHT    | 0.15495104173859986 
 R_O2Sat    | 25.100

In [42]:
getNullCounts(bottle).show()

+------+------+----------+-------+-------+-----+-----+------+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2Sat|R_NO3|R_NO2|R_CHLA|R_PHAEO|
+------+------+----------+-------+-------+-----+-----+------+-------+
|     0|     0|         0|      0|      0|    0|    0|     0|      0|
+------+------+----------+-------+-------+-----+-----+------+-------+



***
[(Return to Table of Contents)](#toc)

# Feature Selection and Target Cleaning<a id="featuretarget"></a>

### Distribution of data for cholorophyll levels, summary stats<a id="summarystats"></a>

In [43]:
bottle.agg(min('R_CHLA'), max('R_CHLA'), mean('R_CHLA'), stddev('R_CHLA'), count('R_CHLA'), skewness('R_CHLA')).show()

+-----------+-----------+-------------------+-------------------+-------------+------------------+
|min(R_CHLA)|max(R_CHLA)|        avg(R_CHLA)|stddev_samp(R_CHLA)|count(R_CHLA)|  skewness(R_CHLA)|
+-----------+-----------+-------------------+-------------------+-------------+------------------+
|      -0.01|      66.11|0.45214100309038985|  1.214949256785272|       222293|10.197710315464647|
+-----------+-----------+-------------------+-------------------+-------------+------------------+



Looking at this summary data of our chlorophyll levels, the data is highly positively skewed. This is displayed in our skewness value but also the difference between the maximum value and the mean.

### Define cholorophyll category levels: low, medium, high?<a id="lowmediumhigh"></a>

As displayed in Figure 4 of [this](https://www.nature.com/scitable/knowledge/library/the-biological-productivity-of-the-ocean-70631104/) article, the chlorophyll concentrations in the ocean can be split into three category levels, low, medium, and high. Our low category range will be between $0 > x > 0.1$μg/L, medium range will be $0.1 \geq x > 1$μg/L, and high concentrations will be $x \geq 1$μg/L.

### Map cholorophyll levels<a id="maplevels"></a>

Map cholorophyll levels to defined categories and output graphs/summary stats and Add target column with category label to dataframe

In [44]:
lowThreshold = .1
medThreshold = 1

In [45]:
bottle = bottle.withColumn(
    'Target',
     when((col('R_CHLA') < lowThreshold), 0)\
    .when((col('R_CHLA').between(lowThreshold, medThreshold)), 1)\
    .when((col('R_CHLA') > medThreshold), 2)\
    .otherwise(10)
)

In [46]:
bottle.groupby('Target').count().show()

+------+------+
|Target| count|
+------+------+
|     1|121991|
|     2| 19492|
|     0| 80810|
+------+------+



In [47]:
bottle = bottle.drop('R_CHLA')

***
[(Return to Table of Contents)](#toc)

# Models<a id="models"></a>

#### Assemble Feature Vector<a id="assemble"></a>

In [48]:
# Checking schema of data again. Using bottleTest df instead of 'bottle' because it has no null values
#  and used vector assembler to create feature vector.
bottle.printSchema()

root
 |-- S_prec: integer (nullable = true)
 |-- R_TEMP: double (nullable = true)
 |-- R_SALINITY: double (nullable = true)
 |-- R_DYNHT: double (nullable = true)
 |-- R_O2Sat: double (nullable = true)
 |-- R_NO3: double (nullable = true)
 |-- R_NO2: double (nullable = true)
 |-- R_PHAEO: double (nullable = true)
 |-- Target: integer (nullable = false)



In [49]:
features = bottle.columns
features.remove('Target')
bottle = VectorAssembler(outputCol='features_unscaled').setInputCols(features).transform(bottle)

In [50]:
bottle.show(2, truncate=False)

+------+------+----------+-------+-------+-----+-------------------+-------+------+----------------------------------------------------------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2Sat|R_NO3|R_NO2              |R_PHAEO|Target|features_unscaled                                         |
+------+------+----------+-------+-------+-----+-------------------+-------+------+----------------------------------------------------------+
|3     |19.23 |34.491    |0.0    |103.9  |1.3  |0.05597931333790767|0.47   |1     |[3.0,19.23,34.491,0.0,103.9,1.3,0.05597931333790767,0.47] |
|3     |19.22 |34.492    |0.03   |103.9  |1.9  |0.05597931333790767|0.38   |1     |[3.0,19.22,34.492,0.03,103.9,1.9,0.05597931333790767,0.38]|
+------+------+----------+-------+-------+-----+-------------------+-------+------+----------------------------------------------------------+
only showing top 2 rows



#### Split Data: Train, Test, Validation, 80, 10, 10<a id="split"></a>

In [51]:
seed = 42
train, test, validation = bottle.randomSplit([0.80, 0.10, 0.10], seed=seed)
print('Train dataset count:', train.count())
print('Test dataset count:', test.count())
print('Validation dataset count:', validation.count())

Train dataset count: 177856


Test dataset count: 22091


Validation dataset count: 22346


#### Scale Data<a id="scale"></a>

In [52]:
from pyspark.ml.feature import MinMaxScaler

mmScaler = MinMaxScaler(inputCol='features_unscaled', outputCol='features')
mmModel = mmScaler.fit(train)

In [53]:
trainscaled = mmModel.transform(train)
testscaled = mmModel.transform(test)
valscaled = mmModel.transform(validation)

#### Check for Imblance in datasets<a id="checkimbalance"></a>

In [54]:
# Ensuring that datasets are balanced with proportional amounts of targets in each set.

print(trainscaled.groupby('target').count().show())
print(testscaled.groupby('target').count().show())
print(valscaled.groupby('target').count().show())

+------+-----+
|target|count|
+------+-----+
|     1|97554|
|     2|15536|
|     0|64766|
+------+-----+

None


+------+-----+
|target|count|
+------+-----+
|     1|12154|
|     2| 1973|
|     0| 7964|
+------+-----+

None


+------+-----+
|target|count|
+------+-----+
|     1|12283|
|     2| 1983|
|     0| 8080|
+------+-----+

None


***
[(Return to Table of Contents)](#toc)

## Logistic Regresssion Model<a id="logisticregression"></a>

#### Testing parameters<a id="lrparams"></a>

In [55]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [56]:
# Testing paramteters to optimize model

regParams = [0.0001, 0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01,
             0.05, 0.1, 0.5, 1.0, 5.0, 10.0]
elasticNetParams = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

results = []
for r in regParams:
    for e in elasticNetParams:
        lr = LogisticRegression(featuresCol='features', labelCol='Target', predictionCol='prediction',
                                maxIter=10, regParam=r, elasticNetParam=e)
        lrModel = lr.fit(trainscaled)
        predictVal = lrModel.evaluate(valscaled)
        results.append([r, e, predictVal.accuracy, predictVal.weightedFMeasure()])

In [57]:
resultsDF = spark.createDataFrame(data=results,
                                  schema = ['regParam', 'elasticNetParam', 'Validation Accuracy', 'Validation F Score'])
resultsDF.show(5)

+--------+---------------+-------------------+------------------+
|regParam|elasticNetParam|Validation Accuracy|Validation F Score|
+--------+---------------+-------------------+------------------+
|  1.0E-4|            0.1| 0.8721023896894299|0.8695201654144367|
|  1.0E-4|            0.2| 0.8752796921149198|0.8746723325514909|
|  1.0E-4|            0.3| 0.8752796921149198|0.8746723325514909|
|  1.0E-4|            0.4| 0.8752349413765327|0.8746279151837562|
|  1.0E-4|            0.5| 0.8751901906381455|0.8745856863175521|
+--------+---------------+-------------------+------------------+
only showing top 5 rows



#### Fit the model with optimal parameters<a id="lrfit"></a>

In [58]:
# Fitting model with optimal parameters

lr = LogisticRegression(featuresCol='features', labelCol='Target', predictionCol='prediction', \
                        maxIter=10, regParam=0.001, elasticNetParam=1.0)
lrModel = lr.fit(trainscaled)

print('LR Model Coefficients: \n' + str(lrModel.coefficientMatrix))
print('LR Model Intercepts: ' + str(lrModel.interceptVector))

accuracy = lrModel.summary.accuracy
falsePositiveRate = lrModel.summary.weightedFalsePositiveRate
truePositiveRate = lrModel.summary.weightedTruePositiveRate
fMeasure = lrModel.summary.weightedFMeasure()
precision = lrModel.summary.weightedPrecision
recall = lrModel.summary.weightedRecall

print('Train Accuracy: %s\nFalse Positive Rate: %s\nTrue Positive Rate: %s\nF-measure: %s\nPrecision: %s\nRecall: %s'
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

LR Model Coefficients: 
DenseMatrix([[ 8.13439736e-01,  1.20235283e+00, -5.55313704e-01,
               8.60305846e+00, -6.91012581e+00,  4.00840013e+00,
              -3.01977876e+01, -3.00305050e+02],
             [ 3.02323948e-01, -0.00000000e+00, -7.19445285e+00,
               1.94455926e+00, -1.02259362e-01, -1.80888893e+00,
               1.50306950e+01,  4.77062076e+00],
             [-1.32808044e+00, -1.57841165e+00,  2.99381542e+00,
              -1.69140813e+01,  8.98587529e+00, -1.55633088e+00,
               1.62935896e+01,  1.69828764e+02]])
LR Model Intercepts: [45.8754073876884,-3.3363947339728597,-42.53901265371553]


Train Accuracy: 0.8746963835912198
False Positive Rate: 0.12600714446134506
True Positive Rate: 0.8746963835912199
F-measure: 0.8728389376460775
Precision: 0.8775886832480038
Recall: 0.8746963835912199


#### Results<a id="lrresults"></a>

In [59]:
# After tuning, print results on test dataset

predictTest = lrModel.evaluate(testscaled)
accuracy = predictTest.accuracy
falsePositiveRate = predictTest.weightedFalsePositiveRate
truePositiveRate = predictTest.weightedTruePositiveRate
fMeasure = predictTest.weightedFMeasure()
precision = predictTest.weightedPrecision
recall = predictTest.weightedRecall

print('Test Results')
print('Accuracy: %s\nFalse Positive Rate: %s\nTrue Positive Rate: %s\nF-measure: %s\nPrecision: %s\nRecall: %s'
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Test Results
Accuracy: 0.8743832329908108
False Positive Rate: 0.1268795924507358
True Positive Rate: 0.8743832329908107
F-measure: 0.8724256833444206
Precision: 0.8772589883041413
Recall: 0.8743832329908107


In [60]:
# Find most important features in model per target value

import numpy as np
coeff = np.abs(lrModel.coefficientMatrix.toArray())
coeff = np.argmax(coeff, axis=1)

for i in range(len(coeff)):
    print('For label', i, 'the most important column is', valscaled.columns[coeff[i]])

For label 0 the most important column is R_PHAEO
For label 1 the most important column is R_NO2
For label 2 the most important column is R_PHAEO


***
[(Return to Table of Contents)](#toc)

## Decision Tree<a id="decisiontree"></a>

We now run this decision tree model to answer the questions: Can we increase accuracy from previous steps, and which features matter the most to this model?

#### Testing parameters<a id="dtparams"></a>

In [61]:
from pyspark.ml.classification import DecisionTreeClassifier

In [62]:
def calculate_accuracy(df):
    correct = df.filter(df['Target'] == df['prediction']).count()
    totalRows = df.count()
    accuracy = correct/totalRows
    return accuracy

In [63]:
splits = [20000, 10000, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 50, 10]

results = []
for s in splits:
    dt = DecisionTreeClassifier(labelCol='Target', seed=seed, minInstancesPerNode=s)
    dtModel = dt.fit(trainscaled)
    dtModel.setFeaturesCol('features')
    trainResult = dtModel.transform(trainscaled)
    valResult =  dtModel.transform(valscaled)
    trainAccuracy = calculate_accuracy(trainResult)
    valAccuracy = calculate_accuracy(valResult)
    results.append([s, trainAccuracy, valAccuracy])

In [64]:
resultsDF = spark.createDataFrame(data=results, schema = ['Splits', 'Train Accuracy', 'Validation Accuracy'])
resultsDF.show()

+------+------------------+-------------------+
|Splits|    Train Accuracy|Validation Accuracy|
+------+------------------+-------------------+
| 20000|0.7890765563152213|  0.788910767027656|
| 10000|0.8322744242533285|  0.837733822608073|
|  1000|0.8700128193594818| 0.8737581670097556|
|   900|0.8702039852464916| 0.8742056743936275|
|   800|  0.87085619827276| 0.8746531817774993|
|   700|  0.87085619827276| 0.8746531817774993|
|   600|0.8723180550557754| 0.8754586950684686|
|   500|0.8725879363080244| 0.8754586950684686|
|   400|0.8731558114429651| 0.8756376980220174|
|   300|  0.87340320259086| 0.8762194576210508|
|   200|0.8735493882691616| 0.8766222142665354|
|   100|0.8733132421734436| 0.8766222142665354|
|    50|0.8733132421734436| 0.8766222142665354|
|    10|0.8733132421734436| 0.8766222142665354|
+------+------------------+-------------------+



#### Fit the model with optimal parameters<a id="dtfit"></a>

In [65]:
# Fitting model with optimized parameter
s = 300
dt = DecisionTreeClassifier(labelCol='Target', seed=seed, minInstancesPerNode=s)
dtModel = dt.fit(trainscaled)
dtModel.setFeaturesCol('features')
testResult = dtModel.transform(testscaled)
testAccuracy = calculate_accuracy(testResult)
print('Decision Tree Accuracy Score: ', testAccuracy)

Decision Tree Accuracy Score:  0.8729346792811552


#### Results<a id="dtresults"></a>

In [66]:
print('Model Depth: ', dtModel.depth)
print('Model Number of Nodes: ', dtModel.numNodes)

Model Depth:  5
Model Number of Nodes:  37


In [67]:
print('Feature Importance: ', dtModel.featureImportances)

Feature Importance:  (8,[1,3,4,7],[0.6096191254066429,0.030217862489867225,0.010688923022960848,0.3494740890805291])


The indeces [1,3,4,5,7] align with ['R_TEMP', 'R_DYNHT', 'R_O2Sat', 'R_NO3', 'R_PHAEO'], meaning these are the most important features (in order) to the decision tree model.

# Stop spark session<a id="stop"></a>

In [68]:
spark.stop()